In [ ]:
# Passsing only existing steps in prompt to the model
# Working

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import AzureChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate
import openai
import openai
import re

class ScenarioGenerator:
    def __init__(self, azure_api_key, azure_endpoint, deployment_name):
        self.azure_api_key = azure_api_key
        self.azure_endpoint = azure_endpoint
        self.deployment_name = deployment_name
        
        openai.api_type = "azure"
        openai.api_base = azure_endpoint
        openai.api_version = "2024-02-15-preview"
        openai.api_key = azure_api_key
        
        self.llm = AzureChatOpenAI(
            openai_api_key=azure_api_key,
            openai_api_base=azure_endpoint,
            openai_api_version="2024-02-15-preview",
            deployment_name=deployment_name,
            temperature=0.3
        )
        
        self.embeddings = OpenAIEmbeddings(
            model="text-embedding-ada-002",
            openai_api_key=azure_api_key,
            openai_api_base=azure_endpoint,
            openai_api_type="azure",
            openai_api_version="2024-02-15-preview",
            deployment="text-embedding-ada-002"
        )
        
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
        )
    
    def generate_scenario(self, scenario_description, existing_steps_file):
        with open(existing_steps_file, 'r', encoding='utf-8') as file:
            content = file.read()
            
        texts = self.text_splitter.split_text(content)
        vectorstore = FAISS.from_texts(texts, self.embeddings)
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", "You are an AI assistant that provides accurate and concise responses."),
            ("human", """
            here is my existing scenario cucumber steps, create another cucumber scenario reusing the steps which I already have, feel free to modify any variable in the step if needed. For any step, if its not defined in the provided scenario cucumber steps, create a new step and add a comment tag "NEW STEP" right before that line.

            Create new scenario for:
            {question} 

            existing scenario cucumber steps:
            {context}
            """)
        ])
        
        scenario_chain = create_stuff_documents_chain(
            self.llm,
            prompt,
            document_variable_name="context"
        )
        
        docs = vectorstore.similarity_search(scenario_description, k=5)
        response = scenario_chain.invoke({
            "question": scenario_description,
            "context": docs
        })
        
        return response


class StepDefinitionGenerator:
    def __init__(self, azure_api_key, azure_endpoint, deployment_name):
        self.azure_api_key = azure_api_key
        self.azure_endpoint = azure_endpoint
        self.deployment_name = deployment_name
        
        openai.api_type = "azure"
        openai.api_base = azure_endpoint
        openai.api_version = "2024-02-15-preview"
        openai.api_key = azure_api_key
    
    def extract_new_steps(self, generated_scenario):
        new_steps = []

        # with open("all_steps.txt", 'r', encoding='utf-8') as file:
        #     content = file.read().split('\n')
        #     content = [line.strip() for line in content if line.strip()]

        # for line in generated_scenario.split('\n'):
        #     pattern = r'^(Given|When|Then|And|But)\s+'
        #     if re.match(pattern, line.strip()):
        #         line = re.sub(pattern, '', line.strip())
        #         if (line not in content):
        #             step = line.strip()
        #             new_steps.append(step)

        save_step = False
        for line in generated_scenario.split('\n'):
            if "NEW STEP" in line:
                save_step = True
                continue
            if save_step:
                new_steps.append(line)
                save_step = False
        exit
        return new_steps
    
    def generate_step_definition(self, step, existing_step_file):
        
        step_pattern = re.sub(r'^(Given|When|Then|And|But)\s+', '', step)
        step_pattern = re.sub(r'"([^"]*)"', '(.*)', step_pattern)
        step_pattern = re.sub(r'<([^>]*)>', '(.*)', step_pattern)
        
        try:
            response = openai.ChatCompletion.create(
                engine=self.deployment_name,
                messages=[
                    {"role": "system", "content": """You are an AI assistant that provides accurate and concise responses.
                    Generate Java step definitions for the provided Cucumber steps.
                    
                    Follow these rules:
                    1. Use Java syntax with Cucumber's step definition patterns
                    2. Include appropriate assertions and verifications
                    3. Handle any parameters or variables in the step
                    4. Include comments explaining complex logic"""},
                    {"role": "user", "content": f"""
                    All Scenario Steps:
                    {existing_step_file}
                    
                    Generate step definition for this step:
                    {step_pattern}
                    """}
                ]
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Error generating step definition: {str(e)}"

def main():
    scenario_gen = ScenarioGenerator(
        azure_api_key="<API_KEY>",
        azure_endpoint="https://cuc2024winterg1743692136.openai.azure.com/",
        deployment_name="gpt-4"
    )
    
    step_def_gen = StepDefinitionGenerator(
        azure_api_key="<API_KEY>",
        azure_endpoint="https://cuc2024winterg1743692136.openai.azure.com/",
        deployment_name="FineTunedCucumberModel"
    )
    
    scenario = """
    Create a preset and use it in creating a new campaign and associate multilingual setting to campaign and publish the campaign
    """
    
    generated_scenario = scenario_gen.generate_scenario(
        scenario,
        "all_steps.txt"
    )
    print(generated_scenario)
    
    new_steps = step_def_gen.extract_new_steps(generated_scenario)
    
    for step in new_steps:
        step_def = step_def_gen.generate_step_definition(
            step, generated_scenario
        )
        print(f"\nStep: {step}")
        print("Step Definition:")
        print(step_def)

if __name__ == "__main__":
    main()

In [ ]:
# Passsing existing steps as well as existing complete sceanrio context in prompt to the model
# Not Complete right now, was not able to test because of exceeded rate limit

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import AzureChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate
import openai
import time

class CucumberStepGenerator:
    def __init__(self, azure_api_key, azure_endpoint, deployment_name):
        self.azure_api_key = azure_api_key
        self.azure_endpoint = azure_endpoint
        self.deployment_name = deployment_name
        
        openai.api_type = "azure"
        openai.api_base = azure_endpoint
        openai.api_version = "2024-02-15-preview"
        openai.api_key = azure_api_key
        
        self.llm = AzureChatOpenAI(
            openai_api_key=azure_api_key,
            openai_api_base=azure_endpoint,
            openai_api_version="2024-02-15-preview",
            deployment_name=deployment_name,
            temperature=0.3
        )
        
        self.embeddings = OpenAIEmbeddings(
            model="text-embedding-ada-002",
            openai_api_key=azure_api_key,
            openai_api_base=azure_endpoint,
            openai_api_type="azure",
            openai_api_version="2024-02-15-preview",
            deployment="text-embedding-ada-002"
        )
        
        self.steps_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
        )

        self.scenario_splitter = RecursiveCharacterTextSplitter(
            separators=["\n\n##"],
            chunk_size=2000,
            chunk_overlap=0,
            length_function=len
        )
        
    def load_contexts(self, steps_file, scenarios_file):
        """Load and process both steps and scenarios files separately"""

        with open(steps_file, 'r', encoding='utf-8') as file:
            steps_content = file.read()
        step_texts = self.steps_splitter.split_text(steps_content)

        self.steps_vectorstore = FAISS.from_texts(step_texts, self.embeddings)
        
        with open(scenarios_file, 'r', encoding='utf-8') as file:
            scenarios_content = file.read()
        scenario_texts = self.scenario_splitter.split_text(scenarios_content)

        self.scenarios_vectorstore = FAISS.from_texts(scenario_texts, self.embeddings)
        
        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an AI assistant that provides accurate and concise responses."""),
            ("human", """
            I will provide you with two types of context:
            1. Individual cucumber steps that can be reused
            2. Complete existing scenarios for reference
            
            Create a new scenario reusing existing steps where possible.
            For any step that doesn't exist in the provided steps, create a new step 
            and mark it with '#completely new step'.

            Create new scenario for:
            {question}

            Available individual steps for reuse:
            {steps_context}

            Example existing scenarios for reference:
            {scenarios_context}

            Make sure to:
            1. Follow the same pattern as existing scenarios
            2. Reuse existing steps where possible
            3. Only mark truly new steps with #completely new step
            4. Maintain the correct Given/When/Then/And order
            """)
        ])

        
        self.qa_chain = create_stuff_documents_chain(
            self.llm,
            prompt,
            document_variable_name="context"
        )
        
    def generate_steps(self, scenario_description):
        """Generate Cucumber steps for a new scenario"""

        relevant_steps = self.steps_vectorstore.similarity_search(scenario_description, k=3)
        relevant_scenarios = self.scenarios_vectorstore.similarity_search(scenario_description, k=2)
        
        steps_text = "\n".join(doc.page_content for doc in relevant_steps)

        scenarios_text = "\n".join(doc.page_content for doc in relevant_scenarios)

        response = self.qa_chain.invoke({
            "question": scenario_description,
            "steps_context": steps_text,
            "scenarios_context": scenarios_text
        })
        
        return response

def main():
    generator = CucumberStepGenerator(
        azure_api_key="<API_KEY>",
        azure_endpoint="https://cuc2024winterg1743692136.openai.azure.com/",
        deployment_name="gpt-4"
    )
    
    generator.load_contexts("all_steps.txt", "all_scenarios.txt")
    
    scenario = """
    Create a campaign using email channel and associate multilingual setting to it and publish the campaign
    """
    
    generated_steps = generator.generate_steps(scenario)
    print("\nGenerated Steps:")
    print(generated_steps)

if __name__ == "__main__":
    main()